### Part 1: 將 HDR file 的 E 從 8-bit clip 到 6-bit

In [3]:
import numpy as np

In [41]:
def read_hdr_rgbe(path):
    with open(path, "rb") as f:
        while True:
            line = f.readline().decode(errors="ignore")
            if line.strip()=="":
                break

        line=f.readline().decode().strip().split()
        H=int(line[1])
        W=int(line[3])
        print(f"Width: {W}, Height: {H}")

        img=np.zeros((H,W,4),dtype=np.uint8)

        for y in range(H):
            header=f.read(4)
            if header[0]!=2 or header[1]!=2:
                raise ValueError("Not RLE Radiance HDR")

            scan = np.zeros((W,4),dtype=np.uint8)
            for c in range(4):
                x=0
                while x<W:
                    val=ord(f.read(1))
                    if val>128:   # run
                        cnt=val-128
                        b=ord(f.read(1))
                        scan[x:x+cnt,c]=b
                        x+=cnt
                    else:         # literal
                        raw=f.read(val)
                        scan[x:x+val,c]=list(raw)
                        x+=val
            img[y]=scan
    return img,W,H

In [42]:
def clamp_E_4bit_signed(hdr):
    hdr2=hdr.copy().astype(np.int16)

    E = hdr2[:,:,3]                             # 0~255
    Esigned = E - 128                           # → -128 ~ +127
    Eclamp = np.clip(Esigned, -8, 7)            # → 限制到 4-bit signed
    Enew = (Eclamp + 128).astype(np.uint8)      # → 回到 RGBE 格式

    hdr2[:,:,3] = Enew
    return hdr2.astype(np.uint8)

In [43]:
def write_hdr_rgbe(path,img,W,H):
    with open(path,"wb") as f:
        f.write(b"#?RADIANCE\nFORMAT=32-bit_rle_rgbe\n\n")
        f.write(f"-Y {H} +X {W}\n".encode())

        for y in range(H):
            f.write(bytes([2,2,(W>>8)&255,W&255]))
            scan=img[y]

            for c in range(4):
                x=0
                while x<W:
                    # RLE run block
                    run=1
                    while x+run<W and scan[x+run,c]==scan[x,c] and run<127:
                        run+=1

                    if run>=2:
                        f.write(bytes([128+run,scan[x,c]]))
                        x+=run
                    else:
                        start=x
                        x+=1
                        while x<W and scan[x,c]!=scan[x-1,c] and (x-start)<127:
                            x+=1
                        block=scan[start:x,c].tolist()
                        f.write(bytes([len(block)]))
                        f.write(bytes(block))

    print("✔ saved:",path)

In [45]:
hdr,W,H = read_hdr_rgbe("../hdr_file/Desk.hdr")

hdr4 = clamp_E_4bit_signed(hdr)

write_hdr_rgbe("../hdr_clip/Ocean.hdr", hdr4, W,H)

Width: 644, Height: 874
✔ saved: ../hdr_clip/Ocean.hdr


### Part 2: 將轉好的 HDR 檔案 的 RGBE 資訊 (共 8*3 + 4 = 28 bits) 寫入 dat file 讓 testbench 存入 sram 內 
格式為 R(8) G(8) B(8) E(4) 用 hex 保存 

In [33]:
def rgbe_to_28bit_4bitE_safe(rgbe):
    """
    RGBE array of uint8 (R,G,B,E) → 28-bit int
    E 已經 clip 到 4-bit signed
    """
    # np.uint8 轉 int
    R = int(rgbe[0].item())
    G = int(rgbe[1].item())
    B = int(rgbe[2].item())
    E4 = (int(rgbe[3].item()) - 128) & 0xF

    val = (R << 20) | (G << 12) | (B << 4) | E4
    return val

In [38]:
def write_dat_4bytes(hdr4, filename):
    """
    hdr4: H x W x 4 uint8 RGBE，E 已 clip 到 4-bit signed範圍
    filename: 輸出的 .dat 檔
    每個 pixel 4 bytes (R,G,B,E)，直接 hex 寫入，每行一個 pixel
    """
    H, W, _ = hdr4.shape
    with open(filename, "w") as f:
        for y in range(H):
            for x in range(W):
                pixel = hdr4[y, x]

                # 直接用 4 個 f.write (hex)
                f.write("{:02X}".format(int(pixel[0].item())))  # R
                f.write("{:02X}".format(int(pixel[1].item())))  # G
                f.write("{:02X}".format(int(pixel[2].item())))  # B
                f.write("{:01X}".format(int(pixel[3].item())-120))  # E
                f.write("\n")  # 每行一個 pixel
    print("✔ DAT file saved:", filename)

In [39]:
dat_path = "../dat_file/input_pattern_0.dat"
write_dat_4bytes(hdr4, "../dat_file/input_pattern_0.dat")

✔ DAT file saved: ../dat_file/input_pattern_0.dat


### Part 3: 固定長寬